In [1]:
import numpy as np
import pandas as pd
import dash
from dash import dash_table
from dash import dcc
import plotly.figure_factory as ff
from dash import html
from dash.dependencies import Input, Output
from sqlalchemy import create_engine
import psycopg2
import os

POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import dash_bootstrap_components as dbc

from bson.json_util import loads, dumps

In [2]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'artipy'
))

In [3]:
impressionism_art = '''
Impressionism  emerged in France during the late 1800's. Rather than focusing on detailed realism, Impressionist artists sought to convey the atmosphere, mood, and sensory experience of a scene by including visible brushstrokes, a focusing on the play of light, and emphasizing the overall visual experience rather than creating a detailed representation. 
'''
impressionism_music = '''
Impressionistic elements in music include static harmony, emphasis on instrumental timbres that create a shimmering interplay of “colors”, melodies that lack directed motion, surface ornamentation that obscures or substitutes for melody, and an avoidance of traditional musical form. Composers most often associated with Impressionism include Claude Debussy and Maurice Ravel. 
'''

In [4]:
external_stylesheets = [dbc.themes.LUX]

In [20]:
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        #Stuff on top

        html.H1("Synesthesia:"),
        html.H2("An Exploration of Various Artistic Movements Through Music"),

        #Tabs

        dbc.Tabs([
                    dbc.Tab(label = 'Home', children = [
                    #describe the layout and purpose of project
                    
                    ]),
                    dbc.Tab(label = 'Impressionism', children = [
                    # stuff for impressionism tab goes here
                        #Music box
                        html.Div([
                            #dcc.Markdown(impressionism_music)
                            html.Div(
                                [
                                    html.Div("Change Songs:"),
                                    dbc.Pagination(id="pagination", max_value=25, fully_expanded=False),
                                    html.Img(id='album_image'),
                                    dcc.Markdown(id='song_info'),
                                    html.Audio(id = 'preview_url',
                                       controls=True,  # Add audio controls (play, pause, volume, etc.)
                                       style={'width': '100%'})
                                ]
                            )
                        ], style = {'width': '30%', 'float': 'left'}),
                
                        #Art Gallery
                        html.Div([
                            dcc.Markdown(impressionism_art),
                            html.Audio(src='https://p.scdn.co/mp3-preview/8a26e954352f018cba5d6d0848bb58bf425d6ec0?cid=8ad607b33bc94e5c918b067c1739a6d4',
                                       controls=True,  # Add audio controls (play, pause, volume, etc.)
                                       style={'width': '100%'})
                        ], style = {'width': '68%', 'float': 'right'})
                    
                ]),
                dbc.Tab(label = 'Pop Art', children = [
                    #stuff for pop art tab goes here
                    #Music box
                        html.Div([
                            #dcc.Markdown(impressionism_music)
                            html.Div(
                                [
                                    dbc.Label("Toggle to add songs with similar audio features:"),
                                    dbc.Checklist(
                                        options=[
                                            {"label": "Enhance", "value": 1}
                                        ],
                                        value=[0],
                                        id="switches-input",
                                        switch=True,
                                    ),
                                    html.Div(
                                        [
                                            dcc.Markdown(id="message"),
                                        ]
                                    )
                                ]
                            )
                        ], style = {'width': '30%', 'float': 'left'}),
                
                        #Art Gallery
                        html.Div([
                            dcc.Markdown(impressionism_art),
                            html.Audio(src='https://p.scdn.co/mp3-preview/8a26e954352f018cba5d6d0848bb58bf425d6ec0?cid=8ad607b33bc94e5c918b067c1739a6d4',
                                       controls=True,  # Add audio controls (play, pause, volume, etc.)
                                       style={'width': '100%'})
                        ], style = {'width': '68%', 'float': 'right'})
                ]),
                dbc.Tab(label = 'TBD', children = [
                    #stuff for TBD tab goes here
                    
                ]),
                dbc.Tab(label = 'TBD', children = [
                    #stuff for TBD tab goes here
                    
                ])
            ])

    ]
)

@app.callback([Output('album_image', 'src'),
               Output('song_info', 'children'),
              Output('preview_url', 'src')],
             [Input('pagination', 'active_page')])

def music_box(page):
    if not page:
        page = 1
    
    myquery = '''
    SELECT name, artists_names, album_image, preview_url
           
    FROM spotify
    WHERE first_artist_id = '17hR0sYHpx7VYTMRfFUOmY' 
        OR first_artist_id ='1Uff91EOsvd99rtAupatMP' 
        AND preview_url != 'None'
    LIMIT 25
    ''' 
    df = pd.read_sql_query(myquery, con=engine)

    mymarkdown = f'''
    ### {df['name'][page -1]}
    ##### {df['artists_names'][page -1]}
    '''
    
    return df['album_image'][page -1], mymarkdown, df['preview_url'][page- 1]

# @app.callback([Output(component_id = 'preview_url', component_property = 'src')],
#              [Input(component_id = 'pagination', component_property = 'active_page')])

# def album_image(page):
#     myquery = '''
#     SELECT preview_url
           
#     FROM spotify
#     WHERE first_artist_id = '17hR0sYHpx7VYTMRfFUOmY' 
#         OR first_artist_id ='1Uff91EOsvd99rtAupatMP' 
#         AND preview_url != 'None'
#     ORDER BY popularity
#     LIMIT 10
#     ''' 
#     df = pd.read_sql_query(myquery, con=engine)
#     return[df['preview_url'][page]]

@app.callback(
    Output("message", "children"),
    [Input("switches-input", "value")],
)
def enhanced_value(switches_value):
    if len(switches_value) == 2:
        message = "here"
    else: 
        message = "hello"
    return message

if __name__ == "__main__":
    app.run_server(mode = 'external', host = "0.0.0.0", debug=True)

In [6]:
myquery = '''
SELECT name, album_image
       
FROM spotify
WHERE first_artist_id = '17hR0sYHpx7VYTMRfFUOmY' 
    OR first_artist_id ='1Uff91EOsvd99rtAupatMP' 
    AND preview_url != 'None'
ORDER BY popularity
LIMIT 10
'''

df = pd.read_sql_query(myquery, con=engine)
df

,name,album_image
0,"Préludes, Book I, L. 117: VIII. La fille aux c...",https://i.scdn.co/image/ab67616d00001e02cfb761...
1,"La plus que lente, L. 121",https://i.scdn.co/image/ab67616d00001e02cfb761...
2,Images: No. 3. Rondes de printemps,https://i.scdn.co/image/ab67616d00001e02426245...
3,"Pelléas et Mélisande, L. 88: Act I: Scene I, U...",https://i.scdn.co/image/ab67616d00001e02faca6f...
4,"Préludes, Book 2, L. 123: No. 12, Feux d'artifice",https://i.scdn.co/image/ab67616d00001e0248a15c...
5,"Debussy: Fantaisie for Piano and Orchestra, CD...",https://i.scdn.co/image/ab67616d00001e02e4ddcb...
6,Symphony in B Minor (arr. T. Finno for orchest...,https://i.scdn.co/image/ab67616d00001e02debd4c...
7,"Debussy / Orch. Koechlin: Khamma, CD 132, L. 125",https://i.scdn.co/image/ab67616d00001e02a40b6b...
8,"La boîte à joujoux, L. 128: Épilogue",https://i.scdn.co/image/ab67616d00001e02c54ed9...
9,Images: No. 2. Iberia: Par les rues et par les...,https://i.scdn.co/image/ab67616d00001e02426245...


---------------------------------------------------------------------------
KeyError                                  Traceback (most recent call last)
File /usr/local/lib/python3.11/site-packages/pandas/core/series.py:1007, in Series.__getitem__(
    self=0    https://i.scdn.co/image/ab67616d00001e02cfb...0001e02426245...
Name: album_image, dtype: object,
    key=None
)
   1004     return self._values[key]
   1006 elif key_is_scalar:
-> 1007     return self._get_value(key)
        key = None
        self = 0    https://i.scdn.co/image/ab67616d00001e02cfb761...
1    https://i.scdn.co/image/ab67616d00001e02cfb761...
2    https://i.scdn.co/image/ab67616d00001e02426245...
3    https://i.scdn.co/image/ab67616d00001e02faca6f...
4    https://i.scdn.co/image/ab67616d00001e0248a15c...
5    https://i.scdn.co/image/ab67616d00001e02e4ddcb...
6    https://i.scdn.co/image/ab67616d00001e02debd4c...
7    https://i.scdn.co/image/ab67616d00001e02a40b6b...
8    https://i.scdn.co/image/ab67616d00001e02c5